In [1]:
import pickle
from statsbombpy import sb
from mplsoccer import Pitch
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from one_two.functions import get_pass_data, get_one_twos

In [2]:
# load in all WSL 20/21 passes
com_id = 37
season_id = 90
s = 5
p = 0.75
c = 5
path = ""

comp_df = sb.matches(competition_id=com_id, season_id=season_id)

In [3]:
matches_df = sb.matches(competition_id=37, season_id=90)
match_id1 = matches_df.match_id[2]

df = sb.events(match_id=match_id1)

df

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_won_advantage,foul_won_defensive,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_shot_saved_off_target,goalkeeper_technique,goalkeeper_type,id,index,injury_stoppage_in_chain,interception_outcome,location,match_id,minute,off_camera,out,pass_aerial_won,pass_angle,pass_assisted_shot_id,pass_body_part,pass_cross,pass_cut_back,pass_end_location,pass_height,pass_inswinging,pass_length,pass_outcome,pass_outswinging,pass_recipient,pass_shot_assist,pass_switch,pass_technique,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_outcome,shot_saved_off_target,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b7f68694-0261-4929-ad96-907e62ec630c,1,NaN,NaN,NaN,3775633,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Aston Villa,2647,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'formation': 352, 'lineup': [{'player': {'id'...",Aston Villa,2647,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3474ba9d-a583-4d35-87dc-d06060a70e1c,2,NaN,NaN,NaN,3775633,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Aston Villa,2647,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tottenham Hotspur Women,749,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82665823-49b6-413e-a8bf-68fcc45eab52,3,NaN,NaN,NaN,3775633,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Aston Villa,2647,[dc452a79-0089-4e2e-82b6-d8349fd5718f],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur Women,749,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dc452a79-0089-4e2e-82b6-d8349fd5718f,4,NaN,NaN,NaN,3775633,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,NaN,NaN,NaN,1,Aston Villa,2647,[82665823-49b6-413e-a8bf-68fcc45eab52],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aston Villa,2647,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7afbbb4b-d035-4bee-807a-be4c07f4de1e,1660,NaN,NaN,NaN,3775633,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,From Free Kick,NaN,NaN,NaN,102,Aston Villa,2647,[0f24b6bf-dcc8-4e94-93ce-80d19c9537dc],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aston Villa,2647,00:00:00.000,Half Start,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# get all shot/assist data

def get_key_events(competition_ID, season_ID):
    # load all matches from specified competition and season
    matches_df = sb.matches(competition_id=competition_ID, season_id=season_ID)

    match_ids = matches_df.match_id

    # loop through all matches in season
    key_events = pd.DataFrame([])
    for matchid in match_ids:
        # get event data ...
        df = sb.events(match_id=matchid)

        # now filter for key events ...
        try:
            goal_assist_df = df.dropna(subset=["pass_goal_assist"])
        except Exception:
            goal_assist_df = pd.DataFrame([])

        try:
            shot_assist_df = df.dropna(subset=["pass_shot_assist"])
        except Exception:
            shot_assist_df = pd.DataFrame([])

        shot_df = df[df["type"]=="Shot"]

        # then concat these frames

        key_df = pd.concat([goal_assist_df, shot_assist_df], axis=0)
        key_df = pd.concat([key_df, shot_df], axis=0)

        # store each dataframe of passing data in a dict (key=match id)
        key_events = pd.concat([key_events, key_df], axis=0)
    return key_events

In [ ]:
key_events = get_key_events(competition_ID=com_id, season_ID=season_id)

In [ ]:
# save all key events

with open(f"data/WSL_20_21_key_events.pickle", "wb") as f:
    pickle.dump(key_events, f, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
# get all passes (attempt 2)

all_matches_passes = get_pass_data(com_id, season_id, None, path, all_teams=True)

In [7]:
# get all one-twos per match

one_twos = {}
for key in all_matches_passes.keys():
    one_twos[key] = get_one_twos(all_matches_passes[key], sec_threshold=s, prog_threshold=p, carry_threshold=c)


No one-two passes
No one-two passes
No one-two passes


In [9]:
# save all passes (dictionary)

with open(f"data/WSL_20_21_all_passes.pickle", "wb") as f:
    pickle.dump(all_matches_passes, f, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
# save all one-twos (dictionary)

with open(f"data/WSL_20_21_all_one_twos.pickle", "wb") as f:
    pickle.dump(one_twos, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# key pass ?

# for normal pass in all_passes df

mask = all_passes["id"] in all_one_twos["id"]
not_one_two = all_passes[mask]

not_one_two

